In [8]:
import tkinter as tk
from tkinter import messagebox, Listbox, Scrollbar
import random
import nltk
from nltk.chat.util import Chat, reflections

# Expanded Menu items and prices
MENU = {
    "Sandwich": 50,
    "Burger": 100,
    "Pizza": 150,
    "Fries": 60,
    "Salad": 70,
    "Cake": 80,
    "Ice Cream": 50,
    "Coffee": 30,
    "Tea": 20,
    "Smoothie": 100,
    "Soda": 25,
    "Water": 10
}

# Recommendations based on preferences
RECOMMENDATIONS = {
    "spicy": ["Pizza", "Burger", "Fries"],
    "sweet": ["Cake", "Ice Cream", "Smoothie"],
    "light": ["Salad", "Sandwich", "Tea"]
}

# Special deals
DEALS = {
    "Combo 1": "Burger + Fries + Soda: ₹150",
    "Combo 2": "Pizza + Smoothie: ₹200",
    "Dessert Special": "Cake + Ice Cream: ₹120"
}

# Chatbot conversation pairs for interactivity
pairs = [
    [
        r"hi|hello|hey",
        ["Hello! Welcome to the Cafeteria. How can I assist you today?"]
    ],
    [
        r"(what's on the menu|menu|show menu)",
        ["Here's our menu:\n" + "\n".join([f"{item}: ₹{price}" for item, price in MENU.items()])]
    ],
    [
        r"(recommend|suggest) (.*)",
        ["I recommend trying something %2! Here's a great option: " + random.choice(RECOMMENDATIONS.get("%2", ["Sandwich", "Coffee"]))]
    ],
    [
        r"(.*)(specials|deals|offers)(.*)",
        ["Check out our special deals:\n" + "\n".join([f"{deal}: {offer}" for deal, offer in DEALS.items()])]
    ],
    [
        r"(what do you have for (snacks|drinks|desserts))",
        ["Here are our options:\nSnacks: Sandwich, Burger, Fries, Salad\nDrinks: Coffee, Tea, Smoothie, Soda, Water\nDesserts: Cake, Ice Cream"]
    ],
    [
        r"(what's your favorite|what do you recommend)",
        ["I personally love the Burger with Fries, and for dessert, the Cake is a must-try!"]
    ],
    [
        r"(place order|order|i want to order)(.*)",
        ["Sure! Please use the menu on the right to select your items. I’ll confirm your order in a moment."]
    ],
    [
        r"(thank you|thanks)",
        ["You're welcome! Enjoy your meal!"]
    ],
    [
        r"(quit|exit)",
        ["Goodbye! Have a great day!"]
    ],
    [
        r"(.*)",
        ["I'm sorry, I didn't quite understand that. Please ask about the menu, place an order, or request recommendations."]
    ]
]

# Chatbot instance
chatbot = Chat(pairs, reflections)

# Main chatbot application class
class CafeteriaChatbotApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Interactive Cafeteria Chatbot")
        self.root.geometry("600x550")

        # Chat window
        self.chat_area = tk.Text(self.root, bg="lightgrey", height=12, width=70, state=tk.DISABLED)
        self.chat_area.grid(row=0, column=0, padx=10, pady=10, columnspan=2)
        
        # Scrollbar for chat window
        self.scrollbar = Scrollbar(self.root, command=self.chat_area.yview)
        self.chat_area['yscrollcommand'] = self.scrollbar.set
        self.scrollbar.grid(row=0, column=2, sticky='ns')

        # Entry box for user input
        self.entry_box = tk.Entry(self.root, width=50)
        self.entry_box.grid(row=1, column=0, padx=10, pady=10)

        # Send button
        self.send_button = tk.Button(self.root, text="Send", width=12, command=self.send_message)
        self.send_button.grid(row=1, column=1)

        # Label for the menu
        self.menu_label = tk.Label(self.root, text="Menu (Select items to order):", font=("Arial", 12))
        self.menu_label.grid(row=2, column=0, sticky="w", padx=10)

        # Listbox for displaying menu items
        self.menu_listbox = Listbox(self.root, selectmode=tk.MULTIPLE, height=8)
        for item in MENU.keys():
            self.menu_listbox.insert(tk.END, item)
        self.menu_listbox.grid(row=3, column=0, padx=10, pady=10)

        # Order summary button
        self.order_button = tk.Button(self.root, text="Place Order", command=self.place_order)
        self.order_button.grid(row=4, column=0, sticky="w", padx=10)

        # Clear chat button
        self.clear_button = tk.Button(self.root, text="Clear Chat", width=12, command=self.clear_chat)
        self.clear_button.grid(row=4, column=1)

        # Initialize the chatbot
        self.display_message("Chatbot: Hello! How can I assist you today?\n")

    def display_message(self, message):
        self.chat_area.config(state=tk.NORMAL)
        self.chat_area.insert(tk.END, message + "\n")
        self.chat_area.config(state=tk.DISABLED)
        self.chat_area.see(tk.END)

    def send_message(self):
        user_message = self.entry_box.get().strip()
        if user_message:
            self.display_message(f"You: {user_message}")
            response = chatbot.respond(user_message)
            self.display_message(f"Chatbot: {response}")
        self.entry_box.delete(0, tk.END)

    def place_order(self):
        selected_items = [self.menu_listbox.get(i) for i in self.menu_listbox.curselection()]
        if not selected_items:
            messagebox.showwarning("Order Error", "Please select at least one item from the menu.")
            return

        order_summary = "\n".join([f"{item}: ₹{MENU[item]}" for item in selected_items])
        total_price = sum([MENU[item] for item in selected_items])
        self.display_message(f"Chatbot: You've ordered:\n{order_summary}\nTotal: ₹{total_price}")
        messagebox.showinfo("Order Placed", f"Your order has been placed!\n\n{order_summary}\nTotal: ₹{total_price}")

    def clear_chat(self):
        self.chat_area.config(state=tk.NORMAL)
        self.chat_area.delete(1.0, tk.END)
        self.chat_area.config(state=tk.DISABLED)

# Run the chatbot application
if __name__ == "__main__":
    root = tk.Tk()
    app = CafeteriaChatbotApp(root)
    root.mainloop()
